In [ ]:
import csv
import cv2
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
import os
from sklearn.decomposition import PCA
import math
import joblib

In [ ]:
x_train = pd.read_csv(f'/kaggle/input/tabular-playground-series-aug-2022/train.csv')
x_train = x_train.reset_index().set_index('id').drop('index', axis=1)

# create missing value flag
missing_measurement3_list = []
missing_measurement5_list = []
for i in range(len(x_train['measurement_3'])):
    if np.isnan(x_train['measurement_3'][i]): #element is empty
        missing_measurement3_list.append(0) 
    else:
        missing_measurement3_list.append(1)
x_train['missing_measurement3'] = missing_measurement3_list
for i in range(len(x_train['measurement_5'])):
    if np.isnan(x_train['measurement_5'][i]): #element is empty
        missing_measurement5_list.append(0) 
    else:
        missing_measurement5_list.append(1)
x_train['missing_measurement5'] = missing_measurement5_list


# fill the missing values
features = []
for f in x_train.columns:
    if f == 'loading' or (f.startswith('measurement') and f != 'measurement_17'):
        features.append(f)
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(x_train[features])
x_train[features] = imputer.transform(x_train[features])

# find the correlation among measurement17 and measurement3~9
for product in "A", "B", "C", "D", "E":
    lr = LinearRegression().fit(x_train.loc[(x_train['product_code'] == product) & (x_train.measurement_17.isna() == False), ['measurement_3', 'measurement_4', 'measurement_5','measurement_6', 'measurement_7', 'measurement_8', 'measurement_9']],
                                x_train.loc[(x_train['product_code'] == product) & (x_train.measurement_17.isna() == False), ['measurement_17']])
    x_train.loc[(x_train['product_code'] == product), ['pred_m_17']] = lr.predict(x_train.loc[(x_train['product_code'] == product), ['measurement_3', 'measurement_4', 'measurement_5','measurement_6', 'measurement_7', 'measurement_8', 'measurement_9']])
    for index, row in x_train.iterrows():
        if row["product_code"] == product and math.isnan(row["measurement_17"]):
            x_train.loc[index, 'measurement_17'] = x_train.loc[index, 'pred_m_17']

# split x,y
y_train=x_train.failure
x_train=x_train.drop(['failure'],axis=1)

# create new features
x_train['area'] = x_train['attribute_2'] * x_train['attribute_3']
x_train['average'] = x_train[[f for f in x_train.columns if f.startswith('measurement') and f != 'measurement_17' and f != 'measurement_0' and f != 'measurement_1' and f != 'measurement_2']].mean(axis=1)

# drop the column that is useless
x_train = x_train.drop([f for f in x_train.columns if f.startswith('measurement') and f != 'measurement_17' and f != 'measurement_0' and f != 'measurement_1' and f != 'measurement_2'], axis=1)



# standardize
x_train[['loading', 'measurement_17', 'measurement_0', 'measurement_1','measurement_2',
          'missing_measurement3', 'missing_measurement5']] = StandardScaler().fit_transform(x_train[['loading',
          'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'missing_measurement3', 'missing_measurement5']])
x_train[['area', 'average']] = StandardScaler().fit_transform(x_train[['area', 'average']])

# drop the column that is useless
x_train = x_train.drop(['product_code', 'attribute_3', 'attribute_2', 'attribute_1', 'attribute_0'], axis=1)

In [ ]:
# logistic regression
model = LogisticRegression(penalty='l1', C=0.01, solver='liblinear', random_state=1)
model.fit(x_train, y_train)
joblib.dump(model, 'model')